# Claims classification with Keras: The Python Deep Learning Library

In this notebook, you will train a classification model for claim text that will predict `1` if the claim is an auto insurance claim or `0` if it is a home insurance claim. The model will be built using a type of Deep Neural Network (DNN) called the Long Short-Term Memory (LSTM) recurrent neural network using TensorFlow via the Keras library.

This notebook will walk you through the text analytic process that consists of:

- Example word analogy with Glove word embeddings
- Vectorizing training data using GloVe word embeddings
- Remotely train a LSTM based classifier model on Azure ML Compute
- Monitor training metrics in real-time
- Using the model to predict classifications

## Prepare modules

This notebook will use the Keras library to build and train the classifier.

In [ ]:
import string
import re
import os
import numpy as np
import pandas as pd
import urllib.request

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, optimizers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Embedding, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

print('Keras version: ', keras.__version__)
print('Tensorflow version: ', tf.__version__)

import azureml.core
from azureml.core import Experiment, Workspace, Run, Datastore
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget
from azureml.core.model import Model
from azureml.train.dnn import TensorFlow
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

print("Azure ML SDK version:", azureml.core.VERSION)

**Let's download the pretrained GloVe word embeddings and load them in this notebook.**

This will create a `dictionary` of size **400,000** words, and the corresponding `GloVe word vectors` for words in the dictionary. Each word vector is of size: 50, thus the dimensionality of the word embeddings used here is **50**.

*The next cell might take couple of minutes to run*

In [ ]:
words_list_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
                  'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/glove50d/wordsList.npy')

word_vectors_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
                    'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/glove50d/wordVectors.npy')

word_vectors_dir = './word_vectors'

os.makedirs(word_vectors_dir, exist_ok=True)
urllib.request.urlretrieve(words_list_url, os.path.join(word_vectors_dir, 'wordsList.npy'))
urllib.request.urlretrieve(word_vectors_url, os.path.join(word_vectors_dir, 'wordVectors.npy'))

dictionary = np.load(os.path.join(word_vectors_dir, 'wordsList.npy'))
dictionary = dictionary.tolist()
dictionary = [word.decode('UTF-8') for word in dictionary]
print('Loaded the dictionary! Dictionary size: ', len(dictionary))

word_vectors = np.load(os.path.join(word_vectors_dir, 'wordVectors.npy'))
print ('Loaded the word vectors! Shape of the word vectors: ', word_vectors.shape)

**Create the word contractions map. The map is going to used to expand contractions in our corpus (for example "can't" becomes "cannot").**

In [ ]:
contractions_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
                    'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/glove50d/contractions.xlsx')
contractions_df = pd.read_excel(contractions_url)
contractions = dict(zip(contractions_df.original, contractions_df.expanded))
print('Review first 10 entries from the contractions map')
print(contractions_df.head(10))

## Word analogy example with GloVe word embeddings

GloVe represents each word in the dictionary as a vector. We can use word vectors for predicting word analogies. 

See example below that solves the following analogy: **father->mother :: king->?**

Cosine similarity is a measure used to evaluate how similar two words are. This helper function takes vectors of two words and returns their cosine similarity that range from -1 to 1. For synonyms the cosine similarity will be close to 1 and for antonyms the cosine similarity will be close to -1.

In [ ]:
def cosine_similarity(u, v):
    dot = u.dot(v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    cosine_similarity = dot/norm_u/norm_v
    return cosine_similarity

Let’s review the vector for the words **father**, **mother**, and **king**

In [ ]:
father = word_vectors[dictionary.index('father')]
mother = word_vectors[dictionary.index('mother')]
king = word_vectors[dictionary.index('king')]
print(father)
print('')
print(mother)
print('')
print(king)

To solve for the analogy, we need to solve for x in the following equation:

**mother – father = x - king**

Thus, **x = mother - father + king**

In [ ]:
x = mother - father + king

**Next, we will find the word whose word vector is closest to the vector x computed above**

To limit the computation cost, we will identify the best word from a list of possible answers instead of searching the entire dictionary.

In [ ]:
answers = ['women', 'prince', 'princess', 'england', 'goddess', 'diva', 'empress', 
           'female', 'lady', 'monarch', 'title', 'queen', 'sovereign', 'ruler', 
           'male', 'crown', 'majesty', 'royal', 'cleopatra', 'elizabeth', 'victoria', 
           'throne', 'internet', 'sky', 'machine', 'learning', 'fairy']

df = pd.DataFrame(columns = ['word', 'cosine_similarity'])

# Find the similarity of each word in answers with x
for w in answers:
    sim = cosine_similarity(word_vectors[dictionary.index(w)], x)   
    df = df.append({'word': w, 'cosine_similarity': sim}, ignore_index=True)
    
df.sort_values(['cosine_similarity'], ascending=False, inplace=True)

print(df)

**From the results above, you can observe the vector for the word `queen` is most similar to the vector `x`.**

## Prepare the training data

Contoso Ltd has provided a small document containing examples of the text they receive as claim text. They have provided this in a text file with one line per sample claim.

Run the following cell to download and examine the contents of the file. Take a moment to read the claims (you may find some of them rather comical!).

In [ ]:
data_location = './data'
base_data_url = 'https://databricksdemostore.blob.core.windows.net/data/05.03/'
filesToDownload = ['claims_text.txt', 'claims_labels.txt']

os.makedirs(data_location, exist_ok=True)

for file in filesToDownload:
    data_url = os.path.join(base_data_url, file)
    local_file_path = os.path.join(data_location, file)
    urllib.request.urlretrieve(data_url, local_file_path)
    print('Downloaded file: ', file)
    
claims_corpus = [claim for claim in open(os.path.join(data_location, 'claims_text.txt'))]
claims_corpus

In addition to the claims sample, Contoso Ltd has also provided a document that labels each of the sample claims provided as either 0 ("home insurance claim") or 1 ("auto insurance claim"). This to is presented as a text file with one row per sample, presented in the same order as the claim text.

Run the following cell to examine the contents of the supplied claims_labels.txt file:

In [ ]:
labels = [int(re.sub("\n", "", label)) for label in open(os.path.join(data_location, 'claims_labels.txt'))]
print(len(labels))
print(labels[0:5]) # first 5 labels
print(labels[-5:]) # last 5 labels

As you can see from the above output, the values are integers 0 or 1. In order to use these as labels with which to train our model, we need to convert these integer values to categorical values (think of them like enum's from other programming languages).

We can use the to_categorical method from `keras.utils` to convert these value into binary categorical values. Run the following cell:

In [ ]:
labels = to_categorical(labels, 2)
print(labels.shape)
print()
print(labels[0:2]) # first 2 categorical labels
print()
print(labels[-2:]) # last 2 categorical labels

Now that we have our claims text and labels loaded, we are ready to begin our first step in the text analytics process, which is to normalize the text.

### Process the claims corpus

- Lowercase all words
- Expand contractions (for example "can't" becomes "cannot")
- Remove special characters (like punctuation)
- Convert the list of words in the claims text to a list of corresponding indices of those words in the dictionary. Note that the order of the words as they appear in the written claims is maintained.

Run the next cell to process the claims corpus.

In [ ]:
def remove_special_characters(token):
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_token = pattern.sub('', token)
    return filtered_token

def convert_to_indices(corpus, dictionary, c_map, unk_word_index = 399999):
    sequences = []
    for i in range(len(corpus)):
        tokens = corpus[i].split()
        sequence = []
        for word in tokens:
            word = word.lower()
            if word in c_map:
                resolved_words = c_map[word].split()
                for resolved_word in resolved_words:
                    try:
                        word_index = dictionary.index(resolved_word)
                        sequence.append(word_index)
                    except ValueError:
                        sequence.append(unk_word_index) #Vector for unkown words
            else:
                try:
                    clean_word = remove_special_characters(word)
                    if len(clean_word) > 0:
                        word_index = dictionary.index(clean_word)
                        sequence.append(word_index)
                except ValueError:
                    sequence.append(unk_word_index) #Vector for unkown words
        sequences.append(sequence)
    return sequences

claims_corpus_indices = convert_to_indices(claims_corpus, dictionary, contractions)

**Review the indices of one sample claim**

In [ ]:
print(remove_special_characters(claims_corpus[5]).split())
print()
print('Ordered list of indices for the above claim')
print(claims_corpus_indices[5])
print('')
print('For example, the index of second word in the claims text \"pedestrian\" is: ', dictionary.index('pedestrian'))

**Create fixed length vectors**

The number of words used in a claim, vary with the claim. We need to create the input vectors of fixed size. We will use the utility function `pad_sequences` from `keras.preprocessing.sequence` to help us create fixed size vector (size = 125) of word indices.

In [ ]:
maxSeqLength = 125

X = pad_sequences(claims_corpus_indices, maxlen=maxSeqLength, padding='pre', truncating='post')

print('Review the new fixed size vector for a sample claim')
print(remove_special_characters(claims_corpus[5]).split())
print()
print(X[5])
print('')
print('Lenght of the vector: ', len(X[5]))

### Upload training data to blob store

**Create the `Workspace` from the saved config file**

In [ ]:
import azureml.core

print(azureml.core.VERSION)

from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws)

**Upload features and labels to the default blob store**

In [ ]:
input_location = './inputs'
features_file = os.path.join(input_location, 'features.npy')
labels_file = os.path.join(input_location, 'labels.npy')
os.makedirs(input_location, exist_ok=True)
np.save(features_file, X)
np.save(labels_file, labels)

datastore = ws.get_default_datastore()
datastore.upload(input_location, 
                 target_path = 'inputs', 
                 overwrite = True, 
                 show_progress = True)

# Create a file dataset object that represents the features and the labels files
dataset = Dataset.File.from_files((datastore, 'inputs/*.npy'))

## Remotely Train the DNN using the Azure ML Compute

### Create AML Compute Cluster

Azure Machine Learning Compute is a service for provisioning and managing clusters of Azure virtual machines for running machine learning workloads. Let's create a new Aml Compute in the current workspace, if it doesn't already exist. We will run the model training jobs on this compute target. This will take couple of minutes to create.

In [ ]:
### Create AML CPU based Compute Cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "amlcompute-ad"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6',
                                                           min_nodes=1, max_nodes=1)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current AmlCompute. 
print(compute_target.status.serialize())

### Create the training script

The training script builds and trains the DNN. Review the code below to understand the structure of the neural network. As you will see, in this case we will build a LSTM recurrent neural network. The network will have a word embedding layer that will convert the word indices to GloVe word vectors. The GloVe word vectors are then passed to the LSTM layer, followed by a binary classifier output layer.

Run the next two cells to create and save the training script in the `scripts` folder.

In [ ]:
script_file_folder = './scripts'
script_file_name = 'train.py'
script_file_full_path = os.path.join(script_file_folder, script_file_name)
os.makedirs(script_file_folder, exist_ok=True)

In [ ]:
%%writefile $script_file_full_path
import os
import argparse
import urllib.request
import math
import timeit
import numpy as np
np.random.seed(437)

print("numpy version: {}".format(np.__version__))

import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Model as KModel #resolve name conflict with Azure Model
from tensorflow.keras import layers, optimizers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import Callback

import sklearn
from sklearn.model_selection import train_test_split

from azureml.core import Run
from azureml.core.model import Model

print("keras version: {} tensorflow version: {} sklearn version: {}".format(keras.__version__, 
                                                                            tensorflow.__version__, 
                                                                            sklearn.__version__))

parser = argparse.ArgumentParser("train")
parser.add_argument("--input-folder", type=str, dest='input_folder', help="Reference to inputs")
parser.add_argument('--batch-size', type=int, dest='batch_size', default=16, help='mini batch size for training')
parser.add_argument('--lr', type=float, dest='lr', default=0.0005, help='Learning rate')
parser.add_argument('--epochs', type=int, dest='epochs', default=25, help='Epochs')

args = parser.parse_args()
input_location = args.input_folder
batch_size = args.batch_size
lr = args.lr
epochs = args.epochs

print('Input folder', input_location)
print('Learning rate', lr)
print('Batch size', batch_size)
print('Epochs', epochs)

words_list_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
                  'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/glove50d/wordsList.npy')
word_vectors_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
                    'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/glove50d/wordVectors.npy')
word_vectors_dir = './word_vectors'
os.makedirs(word_vectors_dir, exist_ok=True)
urllib.request.urlretrieve(words_list_url, os.path.join(word_vectors_dir, 'wordsList.npy'))
urllib.request.urlretrieve(word_vectors_url, os.path.join(word_vectors_dir, 'wordVectors.npy'))
dictionary = np.load(os.path.join(word_vectors_dir, 'wordsList.npy'))
dictionary = dictionary.tolist()
dictionary = [word.decode('UTF-8') for word in dictionary]
print('Loaded the dictionary! Dictionary size: ', len(dictionary))
word_vectors = np.load(os.path.join(word_vectors_dir, 'wordVectors.npy'))
print ('Loaded the word vectors! Shape of the word vectors: ', word_vectors.shape)

maxSeqLength = 125
embedding_layer = Embedding(word_vectors.shape[0],
                            word_vectors.shape[1],
                            weights=[word_vectors],
                            input_length=maxSeqLength,
                            trainable=False)

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='sigmoid'))
print(model.summary())

print('Reading input files...')
features_file = os.path.join(input_location, 'features.npy')
labels_file = os.path.join(input_location, 'labels.npy')
X = np.load(features_file)
labels = np.load(labels_file)
print('Done reading input files.')

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=0)

opt = keras.optimizers.Adam(lr=lr)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

run = Run.get_context()
class LogRunMetrics(Callback):
    # callback at the end of every epoch
    def on_epoch_end(self, epoch, log):
        # log a value repeated which creates a list
        run.log('Loss', log['val_loss'])
        run.log('Accuracy', log['val_accuracy'])
_callbacks = [LogRunMetrics()]

print("Model training starting...")
start_time = timeit.default_timer()
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    validation_data=(X_test, y_test),
                    callbacks=_callbacks,
                    verbose=2)
elapsed_time = timeit.default_timer() - start_time
print("Model training completed.")
print('Elapsed time (min): ', round(elapsed_time/60.0,0))

# save the model
os.makedirs('./outputs', exist_ok=True)
model.save(os.path.join('./outputs', 'final_model.hdf5'))
# save training history
with open(os.path.join('./outputs', 'history.txt'), 'w') as f:
    f.write(str(history.history))

print("Model saved in ./outputs folder")
print("Saving model files completed.")

print('Done!')

### Create the ScriptRunConfig with custom TensforFlow 2.0 Enviroment

In this case we pass the following parameters to the training script:

- **input-folder**: Path to the training dataset mounted on the remote compute
- **lr**: Learning rate for the optimizer
- **batch-size**: Training batch size
- **epochs**: Number of training epochs

In [ ]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies
tensorflow_env = Environment.get(workspace=ws, name='AzureML-TensorFlow-2.2-CPU').clone('Custom-TensforFlow-Env')
cd = tensorflow_env.python.conda_dependencies
cd.add_pip_package('numpy==1.18.5')
cd.add_pip_package('scikit-learn==0.23.1')
tensorflow_env.register(workspace=ws)

In [ ]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=script_file_folder, 
                      script=script_file_name, 
                      arguments=['--input-folder', dataset.as_named_input('inputs').as_mount(),
                                 '--lr', 0.001,
                                 '--batch-size', 16,
                                 '--epochs', 100], 
                      compute_target=compute_target, 
                      environment=tensorflow_env)

### Submit the training run

The code pattern to submit a training run to Azure Machine Learning compute targets is always:

- Create an experiment to run.
- Submit the experiment.
- Wait for the run to complete.

In [ ]:
experiment_name = 'claims-classification-exp'
experiment = Experiment(ws, experiment_name)
run = experiment.submit(src)

### Monitor the Run Metrics

Using the azureml Jupyter widget, you can monitor the training run. You can monitor the validation accuracy and validation loss in real-time as the training progresses.

The training will approximately take around 2-4 minutes to complete. Once the training is completed you can then download the trained models locally by running the **Download the trained models** cell.

In [ ]:
RunDetails(run).show()

In the Jupyter widget `Output Logs` drop down, select **azureml-logs/70_driver_log.txt** to review the model training output. Take a look at the final output for the value "val_accuracy". This stands for validation set accuracy. If you think of random chance as having a 50% accuracy, is you model better than random?

It's OK if it's not much better then random at this point- this is only your first model! The typical data science process would continue with many more iterations taking different actions to improve the model accuracy, including:
- Acquiring more labeled documents for training
- Regularization to prevent overfitting
- Adjusting the model hyperparameters, such as the number of layers, number of nodes per layer, and learning rate

### Download the trained model and the model history files

> **Important**: Ensure the training run in the previous cell has completed before running the following cell. You should see a message that the "Run is completed" in the output logs.

In [ ]:
local_outputs = './outputs'
os.makedirs(local_outputs, exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join(local_outputs, f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)

## Test classifying claims

Now that you have constructed a model, try it out against a set of claims.

**Load the trained model**

In [ ]:
from tensorflow.keras.models import load_model
model = load_model(os.path.join(local_outputs, 'final_model.hdf5'))
print('Model loaded!')
print(model.summary())

**Run the following cell to prepare our test data**

In [ ]:
test_claim = ['I crashed my car into a pole.', 
              'The flood ruined my house.', 
              'I lost control of my car and fell in the river.']

test_claim_indices = convert_to_indices(test_claim, dictionary, contractions)
test_data = pad_sequences(test_claim_indices, maxlen=maxSeqLength, padding='pre', truncating='post')

**Now use the model to predict the classification**

Recall the classification model for claim text predicts a value of `1` if the claim is an auto insurance claim or `0` if it is a home insurance claim.

In [ ]:
pred = model.predict(test_data)
pred_label = pred.argmax(axis=1)
pred_df = pd.DataFrame(np.column_stack((pred,pred_label)), columns=['class_0', 'class_1', 'label'])
pred_df.label = pred_df.label.astype(int)
print('Predictions')
pred_df